In [2]:
import numpy as np
import pandas as pd

import serial

import nidaqmx
from nidaqmx.constants import LineGrouping
from nidaqmx.constants import AcquisitionType

import csv
from datetime import datetime
import matplotlib
matplotlib.use("TkAgg")  # Interactive backend for PyCharm/scripts
import matplotlib.pyplot as plt
from time import time, sleep, perf_counter

Teensy 4.1 Voltage Command Setup

In [ ]:
# import all modules and functions
import numpy as np

# Define and connect to Teensy
teensyport = "COM5" #COM-port that Teensy is using
teensy = serial.Serial(teensyport, 115200) #Serial("Port", Bautrate)
teensy.reset_input_buffer() # clear communication buffer containing strings sent from Teensy to Jupyter notebook
teensy.reset_output_buffer() # clear communication buffer containing strings sent from Jupyter notebook to Teensy
# NB: looks like these two functions don't always result in a cleared buffer!
# Alternatively the function "teensy.readlines()" could be used to clear the communication buffer from Teensy to jupyter notebook
teensy.timeout = 1 # max waittime for a string in s
teensy.write_timeout = 5 # max seconds for write action is 5s
print('Teensy connected')

Teensy connected


In [4]:
def teensy_write(str):
    teensy.write((str+'\n').encode()) # send command with correct encoding and termination
    teensy.readline() # first line sent back is repetition of the command sent
    return

def teensy_readline():
    line = teensy.readline() # read line
    line = line.decode() # decode byte to string, i.e. get rid of starting 'b' character
    line = line.strip() # strip termination characters '\r\n'
    return line

def teensy_set_heaters(channel_voltage_pairs):
    """
    Update one or multiple heaters at once
    """
    # send all ChangeSetpoint commands first
    for channel_number, voltage in channel_voltage_pairs:
        command = "ChangeSetpoint " + str(channel_number) + " " + str(voltage)
        teensy_write(command)
        teensy_readline()  # Clear response

    # update setpoints and outputs at once
    teensy_write("UpdateSetpoints")
    teensy_readline()  # Clear response
    
    teensy_write("UpdateOutputs")
    teensy_readline()  # Clear response
    
    # Flush any remaining bytes
    teensy.reset_input_buffer()

    return

Disconnect from Teensy

In [ ]:
#disconnect from Teensy
#teensy.close()

NI 6002 DAQ Setup

<small>Mux switch delay = samples_per_switch / sample_rate</small>

<small> For 1 or 2 channels, maximum is around 24,000 Hz sample rate and 400 samples per switch </small>

<small>Trade off between multiplexer switch time vs standard deviation</small>

In [5]:
sample_rate = 24000       # samples/s
samples_per_switch = 400  # no. of samples per mux channel

In [6]:
# open GLOBAL tasks that continue running
# digital inputs for P0.0-P0.4
dio_task = nidaqmx.Task()
dio_task.do_channels.add_do_chan(
    "Dev2/port0/line0:4",                             # we only need to change P0.0-P0.4
    line_grouping=LineGrouping.CHAN_FOR_ALL_LINES
)

ai_task = nidaqmx.Task()
ai_task.ai_channels.add_ai_voltage_chan("Dev2/ai0")  # voltage
ai_task.ai_channels.add_ai_voltage_chan("Dev2/ai1")  # current

# Configure hardware-timed finite sampling
ai_task.timing.cfg_samp_clk_timing(rate=sample_rate, sample_mode=AcquisitionType.FINITE,samps_per_chan=samples_per_switch)


In [7]:
def select_channel(channel_number):
    if not (1 <= channel_number <= 24):
        raise ValueError("Channel number must be between 1 and 24")
    
    # 5-bit address: [A4, A3, A2, A1, A0]
    bank = (channel_number - 1) // 6
    channel_in_bank = (channel_number - 1) % 6
    addr_bits = [int(b) for b in format(bank, '02b') + format(channel_in_bank, '03b')]  # [A4, A3, A2, A1, A0]

    # Reverse: P0.0 = A0, P0.1 = A1, ..., P0.4 = A4
    bits = addr_bits[::-1]

    # Convert to integer to write to P0.0–P0.4
    value = sum(bit << i for i, bit in enumerate(bits))
    dio_task.write(value, auto_start=True)

    #print(f"Channel {channel_number} selected:")
    #for i, val in enumerate(bits):
        #print(f"  P0.{i}: {'HIGH' if val else 'LOW'} → {3.3 if val else 0} V")
    
    return bits

In [9]:
def calculate_virp(channel_number):
    """
    Reads voltage and current for a specific channel, computes resistance and power.
    
    Args:
        channel_number (int): Channel number (1-24)
    
    Returns:
        v (float): voltage in V
        i (float): current in A
        r (float): resistance in Ohms
        p (float): power in W
    """
    # Select the channel
    select_channel(channel_number)
    
    # Wait 1 ms for multiplexer to settle
    sleep(0.001)
    
    # Start task, read samples, stop task
    ai_task.start()
    data = ai_task.read(number_of_samples_per_channel=samples_per_switch)
    ai_task.stop()
    
    # Separate channel data
    voltage_samples = np.array(data[0])
    current_samples = np.array(data[1])
    
    # Compute mean values for stable readings
    v = 10 * np.mean(voltage_samples)      # hardware voltage divider
    i = np.mean(current_samples) / 0.5     # sense resistor = 0.5 Ohms
    r = v / i if i != 0 else float('inf')
    p = v * i
    
    return v, i, r, p

In [14]:
def measure_channels(selected_channels=None, delay=0.001):
    """
    Switches multiplexer channel, waits for settling, then takes measurements.
    
    Args:
        selected_channels (list): List of channel numbers (1-24). Defaults to all 24.
        delay (float): Settling time in seconds after channel switch.
    
    Returns:
        data (list): List of tuples [(channel, voltage, current, resistance, power), ...]
    """
    
    # default to all 24 channels
    if selected_channels is None:
        selected_channels = list(range(1, 25))
    
    data = []
    
    for ch in selected_channels:
        # validate channel number
        if not (1 <= ch <= 24):
            print(f"Warning: Invalid channel {ch}. Skipping.")
            continue
        
        # take measurements (calculate_virp handles channel selection)
        v, i, r, p = calculate_virp(ch)
        data.append((ch, v, i, r, p))
    
    return data

In [15]:
def initialise_csv(channel_numbers, filename=None):

    '''
    Create CSV file with appropriate headers to log multi-channel data.
    Args:
        channel_numbers (list)
        filename (str, optional): to reference CSV file
    '''

    # create timestamped CSV file
    filename = f"singleheater_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

    # create headers
    headers = ["Time (s)"] # first column is always time

    # for each channel, add 4 measurement columns
    for ch in channel_numbers:
        headers.extend([
            f"Ch{ch}_Voltage (V)", 
            f"Ch{ch}_Current (A)", 
            f"Ch{ch}_Resistance (Ω)", 
            f"Ch{ch}_Power (W)"
        ])
        
    # create new CSV file with mode = 'w', and write headers
    with open(filename, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        
    print(f"CSV file initialised: {filename}")
    return filename

In [16]:
def save_data_to_csv(data, filename, channel_numbers):
    '''
    Appends latest data point from each channel and 
    writes it as single row to CSV file.
    Args:
        data (dict): Nested dictionary containing data:
        {
            "times": [...],
            "ch_1": {"voltages": [...], "currents": [...], ...},
            "ch_2": {"voltages": [...], "currents": [...], ...},
            ...
        }
        filename (str): path to CSV file to append data to
        channel_numbers (list)

    '''
    with open(filename, mode= "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)

        # if we have time data (measurements exist), build row with timestamp
        if data["times"]:
            latest_time = data["times"][-1]
            row = [latest_time]

            # get most recent measurements for each channel            
            for ch in channel_numbers:

                # get channel's data dictionary
                ch_data = data[f"ch_{ch}"]

                # append only most recent measurements
                row.extend([
                    ch_data["voltages"][-1],
                    ch_data["currents"][-1], 
                    ch_data["resistances"][-1],
                    ch_data["powers"][-1]
                    ])
                
            # write complete row to CSV file
            writer.writerow(row)

In [17]:
def initialise_plots(channel_numbers):
    '''
    create individual plot for single channel or overlay plots for multiple channels
    Args:
    channel_numbers (list)
    Returns:
        tuple: (fig, axs, data, lines) where:
            fig: matplotlib figure object
            axs: array of subplot axes, always 4 subplots in 2x2 grid
            data: nested dictionary
            lines: matplotlib line objects
    '''
    plt.ion()  # Enable interactive mode for real-time updates
    
    n_channels = len(channel_numbers)
    
    if n_channels == 1:
        # Single channel: 4 subplots
        fig, axs = plt.subplots(2, 2, figsize=(10, 8))
        axs = axs.ravel()  # flatten 2D array to 1D for easy indexing
    else:
        # Multiple channels: overlay all channels on same 4 subplots
        fig, axs = plt.subplots(2, 2, figsize=(12, 8))
        axs = axs.ravel()  # flatten 2D array to 1D for easy indexing
    
    # initiliase data structures to store measurements
    data = {"times": []}
    lines = {}
    
    colors = ['r-', 'g-', 'b-', 'm-', 'c-', 'y-', 'k-', 'orange', 'purple', 'brown']
    
    for i, ch in enumerate(channel_numbers):
        # Initialize empty data lists for this channel
        data[f"ch_{ch}"] = {
            "voltages": [],
            "currents": [],
            "resistances": [],
            "powers": []
        }
        
        color = colors[i % len(colors)]
        label = f"Ch{ch}"
        
        # create empty plot lines for each measurement
        # all channels use same 4 subplots
        lines[f"ch_{ch}_voltage"], = axs[0].plot([], [], color, label=label)
        lines[f"ch_{ch}_current"], = axs[1].plot([], [], color, label=label)
        lines[f"ch_{ch}_resistance"], = axs[2].plot([], [], color, label=label)
        lines[f"ch_{ch}_power"], = axs[3].plot([], [], color, label=label)
    
    # Configure subplot appearance and labels
    axs[0].set_title("Voltage", fontsize=10)
    axs[0].set_xlabel("Time (s)", fontsize=10)
    axs[0].set_ylabel("Voltage (V)", fontsize=10)
    axs[0].grid(True, alpha=0.3)
    axs[0].tick_params(axis='both', which='major', labelsize=9)
    
    axs[1].set_title("Current", fontsize=10)
    axs[1].set_xlabel("Time (s)", fontsize=10)
    axs[1].set_ylabel("Current (mA)", fontsize=10)
    axs[1].grid(True, alpha=0.3)
    axs[1].tick_params(axis='both', which='major', labelsize=9)
    
    axs[2].set_title("Resistance", fontsize=10)
    axs[2].set_xlabel("Time (s)", fontsize=10)
    axs[2].set_ylabel("Resistance (Ω)", fontsize=10)
    axs[2].grid(True, alpha=0.3)
    axs[2].tick_params(axis='both', which='major', labelsize=9)
    
    axs[3].set_title("Power", fontsize=10)
    axs[3].set_xlabel("Time (s)", fontsize=10)
    axs[3].set_ylabel("Power (W)", fontsize=10)
    axs[3].grid(True, alpha=0.3)
    axs[3].tick_params(axis='both', which='major', labelsize=9)
    
    # Add legends for multiple channels
    if n_channels > 1:
        for ax in axs:
            ax.legend(fontsize=9)

    # subplot spacing and display
    plt.tight_layout()
    fig.canvas.draw()
    fig.show()

    return fig, axs, data, lines

In [18]:
def update_plots(axs, data, lines, channel_numbers, measurements, last_plot_time, plot_interval=2.0):
    '''
    Updates real-time plots with new measurement data from all channels.
    Only refreshes plots every plot_interval=2.0s to prevent excessive CPU usage.
    Returns:
        last_plot_time (float): updated time of when plots were refreshed
    '''

    # get current time for this measurement cycle
    current_time = time()
    data["times"].append(current_time)   # add time to shared time array

    # store new measurements in data structure for each channel
    for i, ch in enumerate(channel_numbers):
        # extract measurements for channel
        v, i_curr, r, p = measurements[i]

        # get reference to channel's data dictionary
        ch_data = data[f"ch_{ch}"]

        # append new measurements to channel's data arrays
        ch_data["voltages"].append(v)
        ch_data["currents"].append(i_curr)
        ch_data["resistances"].append(r)
        ch_data["powers"].append(p)

   # Always update on first data point (len == 1) or after plot_interval=2.0s
    if current_time - last_plot_time >= plot_interval or len(data["times"]) == 1:

        # 
        for i, ch in enumerate(channel_numbers):    
            ch_data = data[f"ch_{ch}"]   # get channel's data

            # update plot lines with x=times, y=measurements            
            lines[f"ch_{ch}_voltage"].set_data(data["times"], ch_data["voltages"])
            lines[f"ch_{ch}_current"].set_data(data["times"], ch_data["currents"])
            lines[f"ch_{ch}_resistance"].set_data(data["times"], ch_data["resistances"])
            lines[f"ch_{ch}_power"].set_data(data["times"], ch_data["powers"])
            
        # Update axis limits for 4 subplots to accommodate new data
        for ax in axs:
            ax.relim()               # recalculate data limits
            ax.autoscale_view()      # adjust axis ranges to fit all data

        # brief pause for GUI to update and remain responsive
        plt.pause(0.01)

        # update time tracking when plot was last refreshed
        last_plot_time = current_time

    return last_plot_time

In [ ]:
def feedback_loop(channel_numbers, target_resistances, tolerance_resistance,
                  fail_resistance, update_time=2.0, voltage_step=0.1,
                  filename=None, hold_duration=None,
                  fig=None, axs=None, data=None, lines=None):
    
    """
    Continuously adjust voltage to maintain target resistance for one or multiple channels
    Failed channels are set to 0 V and excluded from further control
    All active channels must reach their targets before hold timer starts
    Ctrl + C safely exits and maintain last voltages

    Args:
        channel_numbers (list)
        target_resistances (list)
        tolerance_resistance (float): acceptable deviation from target (±Ω), same for ALL channels
        fail_resistance (float): same for ALL channels
        update_time (float): time interval between measurements and adjustments (s), same for ALL channels
        voltage_step (float): same for ALL channels
        filename (str): set to None for first feedback loop
        hold_duration (float): duration to maintain target(s) before next feedback sequence (s); set to None for continuous operation
        fig, axs, data, lines: Matplotlib objects for continuing real-time plots from previous sequences
    
    Returns:
        last_voltages (dict): voltages (V) for each channel
        last_resistances (dict): resistances (Ω) for each channel
        CSV filename (str): used for subsequent loops
        fig, axs, data, lines: Matplotlib objects for continuing real-time plots
    """
    
    # validate input parameters
    if len(channel_numbers) != len(target_resistances):
        raise ValueError("channel_numbers and target_resistances must have the same length")
    
    # Initialise csv filename for data logging
    # If first call, create new file; otherwise append to existing file fromp previous sequence
    if filename is None:
        filename = initialise_csv(channel_numbers)
    else:
        print(f"Appending data to existing file: {filename}")

    # Initialise or continue real-time plots
    # If first call, create new plots; otherwise append to existing plots from previous sequence
    if fig is None or axs is None or data is None or lines is None:
        plt.ion() # enable interactive plotting mode
        fig, axs, data, lines = initialise_plots(channel_numbers)
    
    # ensure first plot happens immediately
    last_plot_time = 0  

    # dictionaries to track most recent voltage and resistance for each channel
    # persist across loop iterations to enable voltage adjustments
    last_voltages = {}
    last_resistances = {}

    # initialise voltages and resistances for each channel 
    for ch in channel_numbers:
        v, i, r, p = calculate_virp(ch)
        last_voltages[ch]= v
        last_resistances[ch] = 0 # will be updated in first measurement cycle

    # timer for tracking time passed after all channels have maintained target resistances
    # remains None until all channels first reach their target resistances
    hold_start_time = None

    # track which channels have failed
    # once a channel is added, it is excluded from feedback control permanently
    failed_channels_set = set()

    # MAIN FEEDBACK LOOP
    try:
        while True:

            measurements = []
            for ch in channel_numbers:
                v, i, r, p = calculate_virp(ch)
                measurements.append((v, i, r, p))
                last_resistances[ch] = r # update resistance tracking

            last_plot_time = update_plots(axs, data, lines, channel_numbers, measurements, last_plot_time)
            save_data_to_csv(data, filename, channel_numbers)

            # triggered when all channels reach target resistance
            # determines when to start hold timer
            all_targets_reached = True

            # collect all channels requiring voltage updates for batch transmission
            # this is to minimise communication overhead with Teensy
            channels_to_update = []

            # collect all channels that failed and need to be set to 0 V
            failed_channels = []

            for i, ch in enumerate(channel_numbers):
                v_measured, _, r, _ = measurements[i]
                target_r = target_resistances[i]

                # check if channel has exceeded fail resistance
                # trigger only once per channel
                if r > fail_resistance and ch not in failed_channels_set:
                    print(f"Channel {ch} resistance too high. Voltage set to 0 V.")
                    failed_channels.append((ch, 0)) # queue for Teensy batch update
                    last_voltages[ch] = 0 # update voltage tracking
                    failed_channels_set.add(ch) # mark as permanently failed

                # MAIN FEEDBACK CONTROL
                # skip voltage adjustment for failed channels
                elif ch not in failed_channels_set:
                    delta = 0

                    if r > target_r + tolerance_resistance:
                        delta = -voltage_step
                    elif r < target_r - tolerance_resistance:
                        delta = voltage_step

                    # only apply voltage adjustment if needed
                    if delta != 0:
                        new_v = round(last_voltages[ch] + delta, 4)
                        channels_to_update.append((ch, new_v)) # queue for batch update

                        last_voltages[ch] = new_v # update tracked voltage
                        
                        # ensure voltage never drops below 1.1 V (previously used for XDAC)
                        # if new_v < 1.1:
                            # new_v = 1.1
                             
                    # if any active channel is outside tolerance, group condition is False
                    if abs(r - target_r) > tolerance_resistance:
                        all_targets_reached = False
            
            # BATCH VOLTAGE UPDATE - minimise communication overhead and ensure synchronised control
            # send all voltage updates at once to Teensy if there are any changes
            if channels_to_update:
                teensy_set_heaters(channels_to_update)

            # HOLD TIMER MANAGEMENT
            # Start timing after all channels reach target resistance and only if hold_duration is specified
            if all_targets_reached and hold_duration is not None:
                if hold_start_time is None:
                    hold_start_time = time()
                    print("Target resistance reached, starting hold timer...")
                else:
                    # check if hold duration has been reached
                    elapsed = time() - hold_start_time
                    if elapsed >= hold_duration:
                        print(f"Max duration of {hold_duration} s reached. Exiting feedback loop.")
                        break # exit feedback loop, starting the next sequence
            
            # WAIT BEFORE NEXT ITERATION
            sleep(update_time)

    except KeyboardInterrupt:
        print("\nFeedback loop stopped by user.")
        
        # set all channels to last known voltages
        channels_to_reset = [(ch, last_voltages[ch]) for ch in channel_numbers]
        teensy_set_heaters(channels_to_reset)

    # RETURN STATE FOR NEXT SEQUENCE
    return last_voltages, last_resistances, filename, fig, axs, data, lines

In [ ]:
def series_feedback(channel_numbers, feedback_schedule, tolerance_resistance,
                    fail_resistance, filename=None):
    """
    Run a series of feedback loops for multiple target resistances with specified hold durations.
    
    Args:
        channel_numbers (list): list of channel numbers
        feedback_schedule (list of tuples): 
            [([target1_ch1, target1_ch2, ...], hold_duration, update_time, voltage_step), ...]
            The final target should have hold_duration=None to indicate it runs indefinitely.
        tolerance_resistance (float)
        fail_resistance (float)
        filename (str or None)
    
    Returns:
        last_voltages (dict), last_resistances (dict), filename
    """

    last_voltages = {}
    last_resistances = {}

    # initialise plotting objects to keep them across steps
    fig = axs = data = lines = None
    
    for i, (target_resistances, hold_duration, update_time, voltage_step) in enumerate(feedback_schedule):

        # error handling
        if len(target_resistances) != len(channel_numbers):
            raise ValueError(
                f"Step {i+1}: number of target resistances ({len(target_resistances)}) "
                f"does not match number of channels ({len(channel_numbers)})."
            )
        
        print(f"\n--- Step {i+1}: Target resistance = {target_resistances} Ω ---")
        print(f"Update time = {update_time}s, Voltage step = {voltage_step}V")

        # to check if a duration is provided
        if hold_duration is not None:
            print(f"Holding target for {hold_duration} seconds after reaching it...")
        
        else:
            print("Final target: running feedback loop until keyboard interrupt.")

        # Run the feedback loop for this step
        last_voltages, last_resistances, filename, fig, axs, data, lines = feedback_loop(
            channel_numbers=channel_numbers,
            target_resistances=target_resistances,
            tolerance_resistance=tolerance_resistance,
            fail_resistance=fail_resistance,
            update_time=update_time,
            voltage_step=voltage_step,
            filename=filename,
            hold_duration=hold_duration,
            fig=fig,
            axs=axs,
            data=data,
            lines=lines
        )
        
        print(f"Step {i+1} complete. Voltages: {last_voltages}, Resistances: {last_resistances}")

    # show plot at the very end
    plt.ioff()
    plt.show()

    return last_voltages, last_resistances, filename

In [22]:
teensy_set_heaters([(1, 0.2), 
                    (2, 0)])

In [23]:
calculate_virp(1)

(np.float64(0.1947908417212602),
 np.float64(0.005501158242404927),
 np.float64(35.40905989937566),
 np.float64(0.0010715752444799042))

In [24]:
calculate_virp(2)

(np.float64(0.006507465276808943),
 np.float64(0.00021609571631415748),
 np.float64(30.11380969416564),
 np.float64(1.4062353703815358e-06))

In [47]:
channel_numbers = [1, 2]

feedback_schedule = [
    ([50, 45], 30, 0.05, 0.005),
    ([60, 55], None, 0.05, 0.005)      
]

last_voltages, last_resistances, filename = series_feedback(
    channel_numbers=channel_numbers,
    feedback_schedule=feedback_schedule,
    tolerance_resistance=0.05,
    fail_resistance=100,
    filename=None,
)


--- Step 1: Target resistance = [50, 45] Ω ---
Update time = 0.05s, Voltage step = 0.005V
Holding target for 30 seconds after reaching it...
CSV file initialised: singleheater_20251013_134623.csv
Channel 2 resistance too high. Voltage set to 0 V.
Step 1 complete. Voltages: {1: np.float64(5.023), 2: 0}, Resistances: {1: np.float64(40.204433451320234), 2: np.float64(110.52934032144859)}

--- Step 2: Target resistance = [60, 55] Ω ---
Update time = 0.05s, Voltage step = 0.005V
Final target: running feedback loop until keyboard interrupt.
Appending data to existing file: singleheater_20251013_134623.csv
Channel 2 resistance too high. Voltage set to 0 V.
Step 2 complete. Voltages: {1: np.float64(5.5986), 2: 0}, Resistances: {1: np.float64(40.877258004293736), 2: np.float64(106.07383742883312)}


In [ ]:
channel_numbers = [1, 2]

#feedback_schedule (list of tuples): 
    #[([target1_ch1, target1_ch2, ...], hold_duration, update_time, voltage_step), ...]
    #The final target should have hold_duration=None to indicate it runs indefinitely.

feedback_schedule = [
    ([60, 60], 30, 0.5, 0.25), 
    ([58.5, 62.5], 30, 0.5, 0.25),
    ([55, 65], 30, 0.5, 0.25),
    ([52.5, 67.5], 30, 0.5, 0.25),       
    ([50, 70], 30, 0.5, 0.25),       
    ([49, 71], 30, 0.25, 0.125),
    ([48, 72], 30, 0.25, 0.125),
    ([47, 73], 30, 0.25, 0.125),
    ([46, 74], 30, 0.25, 0.125),
    ([45, 75], None, 0.25, 0.125)      
]

last_voltages, last_resistances, filename = series_feedback(
    channel_numbers=channel_numbers,
    feedback_schedule=feedback_schedule,
    tolerance_resistance=0.25,
    fail_resistance=100,
    filename=None,
)


--- Step 1: Target resistance = [60, 60] Ω ---
Update time = 0.5s, Voltage step = 0.25V
Holding target for 30 seconds after reaching it...
CSV file initialised: A13_A14_feedbackcontrol_20250911_114948.csv
Target resistance reached, starting hold timer...
Max duration of 30 s reached. Exiting feedback loop.
Step 1 complete. Voltages: {1: 14.52324, 2: 14.46406}, Resistances: {1: 60.07638174325351, 2: 60.09633072788179}

--- Step 2: Target resistance = [58.5, 62.5] Ω ---
Update time = 0.5s, Voltage step = 0.25V
Holding target for 30 seconds after reaching it...
Appending data to existing file: A13_A14_feedbackcontrol_20250911_114948.csv
Target resistance reached, starting hold timer...
Max duration of 30 s reached. Exiting feedback loop.
Step 2 complete. Voltages: {1: 13.58477, 2: 15.55781}, Resistances: {1: 58.57808297444577, 2: 62.5068954087526}

--- Step 3: Target resistance = [55, 65] Ω ---
Update time = 0.5s, Voltage step = 0.25V
Holding target for 30 seconds after reaching it...
Ap